### Web Scraping with temp function

In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://fangj.github.io/friends/'
res = requests.get(url)

In [3]:
soup = BeautifulSoup(res.content)

In [4]:
links = soup.find_all('a')

In [5]:
a_script =[]
cc=0
script_dict ={}
for l in links:
    st = l.get_text().split(' ')
    if(len(st[0].strip())>=4):
        season = int(st[0][0:2])
        episode = st[0][2:]
    else:
        season = int(st[0][0])
        episode = st[0][1:]
    title = ' '.join(st[1:]).strip()
    surl = url + l['href']
    s_episode = requests.get(surl)
    soup = BeautifulSoup(s_episode.text, 'html.parser')
    dialogues = soup.find_all('p')
    script_start = False
    for d in dialogues:
        diag = d.get_text()
        diag = diag.split(':')
        if(script_start):
            if(len(diag)>1):
                a_script.append([season, episode, title, diag[0],diag[1]])
        elif(len(diag)>1 and diag[0]=='[Scene'):
            script_start = True
    script_dict.update({cc: a_script[cc]})
    cc = cc+1

In [6]:
friends_df = pd.DataFrame(a_script)

In [7]:
friends_df.head()

,0,1,2,3,4
0,1,01,Monica Gets A Roommate,Phoebe,"(singing) Love is sweet as\nsummer showers, l..."
1,1,01,Monica Gets A Roommate,[Scene,"Ross's Apartment, the guys are there assembli..."
2,1,01,Monica Gets A Roommate,Ross,(squatting and reading the instructions) I'm\...
3,1,01,Monica Gets A Roommate,Joey,I'm thinking we've got a\nbookcase here.
4,1,01,Monica Gets A Roommate,Chandler,It's a beautiful thing.


In [10]:
friends_df.rename(columns={0: 'season',
                  1: 'episode',
                  2: 'title',
                  3: 'character',
                  4: 'dialogue'}, inplace=True)

In [11]:
friends_df.head()

,season,episode,title,character,dialogue
0,1,01,Monica Gets A Roommate,Phoebe,"(singing) Love is sweet as\nsummer showers, l..."
1,1,01,Monica Gets A Roommate,[Scene,"Ross's Apartment, the guys are there assembli..."
2,1,01,Monica Gets A Roommate,Ross,(squatting and reading the instructions) I'm\...
3,1,01,Monica Gets A Roommate,Joey,I'm thinking we've got a\nbookcase here.
4,1,01,Monica Gets A Roommate,Chandler,It's a beautiful thing.


In [12]:
friends_df.shape

(58288, 5)

In [14]:
friends_df.to_csv('friends.csv', index=False)